In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as func
from pyspark.sql.functions import col, lit, concat, array, coalesce

from pyspark.ml.linalg import Vectors
from pandas import Series,DataFrame
import pandas as pd
import numpy as np

import ast
import time

## Clean/Prepare Data

In [3]:
lines = sc.textFile("/Users/pudding/Downloads/taxi-data-sorted-small.csv")

In [4]:
taxilines = lines.map(lambda x: x.split(','))

# Exception Handling and removing wrong data lines
def isfloat(value): 
    try :
        float(value)
        return True
    except :
        return False

# Remove lines if they don’t have 16 values and 
# with total amount of larger than 600 USD and less than 1 USD
def correctRows (p) : 
    if(len(p) == 17):
        if(isfloat(p[5]) and isfloat(p[11])): 
            if(float(p[5])!=0 and float(p[11])!=0):
                if(float(p[11])>=1 and float(p[11])<=600):
                    return p

# cleaning up data
texilinesCorrected = taxilines.filter(correctRows)

In [ ]:
texilinesCorrected.saveAsTextFile("/Users/pudding/Downloads/taxi_clean_data")

## Task 1

In [5]:
texilinesCorrected = sc.textFile("/Users/pudding/Downloads/taxi_clean_data")
texilinesCorrected = texilinesCorrected.map(lambda x: ast.literal_eval(x))

In [13]:
column_names = ['medallion', 'hack_license', 'pickup_datetime', 'dropoff_datetime', 
               'trip_time_in_secs', 'trip_distance', 'pickup_longitude', 
               'pickup_latitude', 'dropoff_logitude', 'dropoff_latitude', 
               'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 
               'tip_amount', 'tolls_amount', 'total_amount']

In [14]:
# create the dataframe from RDD
taxiDF = sqlContext.createDataFrame(texilinesCorrected, column_names)

In [15]:
taxiDF.columns

['medallion',
 'hack_license',
 'pickup_datetime',
 'dropoff_datetime',
 'trip_time_in_secs',
 'trip_distance',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_logitude',
 'dropoff_latitude',
 'payment_type',
 'fare_amount',
 'surcharge',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'total_amount']

In [26]:
task1_output = taxiDF.groupby(taxiDF.medallion)\
.agg(func.countDistinct('hack_license')\
.alias('Distinct_Drivers_Count'))\
.sort('Distinct_Drivers_Count', ascending = False)

In [39]:
task1_output.show(n=10)

+--------------------+----------------------+
|           medallion|Distinct_Drivers_Count|
+--------------------+----------------------+
|3C08296D0EB7ABE24...|                    20|
|65EFB7D02BAD12D5D...|                    20|
|7DEB25123AE57111F...|                    19|
|55D311AD2752BC278...|                    19|
|F36564AB9C6EA3B63...|                    19|
|3B6AE3CF05F34ADC9...|                    19|
|CD7B02776E6948339...|                    19|
|9FB7A7C1D7B960D8B...|                    18|
|799153A138F4E8334...|                    18|
|F2A08960199BCDB7E...|                    18|
+--------------------+----------------------+
only showing top 10 rows



## Task 2

In [ ]:
texilinesCorrected.groupBy(lambda x: x[1])

In [32]:
taxiDF_by_Driver = taxiDF.groupBy('hack_license')\
.agg(func.sum('total_amount').alias('all_total_amount'), \
     func.sum('trip_time_in_secs').alias('all_trip_time_in_secs'))

In [33]:
taxiDF_by_Driver.show()

+--------------------+------------------+---------------------+
|        hack_license|  all_total_amount|all_trip_time_in_secs|
+--------------------+------------------+---------------------+
|3183016714F5E253E...|           1675.14|              79020.0|
|31195E1D3AA1EC26D...|           2024.57|             102840.0|
|69996930170E51265...| 2950.609999999999|             147900.0|
|FDEA95843CF14F503...|           3037.15|             154800.0|
|477E608A18D83F7DE...|1885.1499999999999|             107160.0|
|4CEC9B9F46ABAB3AD...|           1835.89|              97980.0|
|CF1BF49F7229C883D...|1948.3100000000002|             111000.0|
|83279FE1F7D548407...|           2803.71|             149280.0|
|7DA853581E8C09723...|2571.8399999999997|             116640.0|
|1FD12923F545A7F28...|           1828.66|              98640.0|
|E7E6F9F97F18C35E0...|2092.7500000000005|             114000.0|
|03A2D28F831C5C3E5...|2309.1600000000003|             115980.0|
|88B5A703C9407B0B3...|2195.0199999999995

In [36]:
taxiDF_by_Driver = taxiDF_by_Driver.withColumn("average_earned_per_minute", \
                                               func.expr("all_total_amount/(all_trip_time_in_secs/60)"))

In [37]:
taxiDF_by_Driver.show()

+--------------------+------------------+---------------------+-------------------------+
|        hack_license|  all_total_amount|all_trip_time_in_secs|average_earned_per_minute|
+--------------------+------------------+---------------------+-------------------------+
|3183016714F5E253E...|           1675.14|              79020.0|       1.2719362186788155|
|31195E1D3AA1EC26D...|           2024.57|             102840.0|        1.181196032672112|
|69996930170E51265...| 2950.609999999999|             147900.0|       1.1970020283975655|
|FDEA95843CF14F503...|           3037.15|             154800.0|       1.1771899224806202|
|477E608A18D83F7DE...|1885.1499999999999|             107160.0|        1.055515117581187|
|4CEC9B9F46ABAB3AD...|           1835.89|              97980.0|       1.1242437232088183|
|CF1BF49F7229C883D...|1948.3100000000002|             111000.0|       1.0531405405405407|
|83279FE1F7D548407...|           2803.71|             149280.0|       1.1268930868167202|
|7DA853581

In [40]:
taxiDF_by_Driver\
.select('hack_license', 'average_earned_per_minute')\
.sort('average_earned_per_minute', ascending = False)\
.show(n=10)

+--------------------+-------------------------+
|        hack_license|average_earned_per_minute|
+--------------------+-------------------------+
|30B2ACBAF00230553...|                    702.0|
|4C3B2A31227663A59...|                    625.0|
|08026D69508127F4D...|                    375.0|
|6E1D7195E38AA7A36...|        317.3076923076923|
|E8E22AC46DF6AC99C...|                    273.0|
|975D5E840C0F5D961...|       190.90909090909093|
|17F72121B9F612D81...|       107.14285714285714|
|619BF4020E6542AA0...|       105.88235294117648|
|CD9D0B4429613F1B6...|       103.25172413793103|
|52C99F4F8CD2560F8...|        89.28571428571429|
+--------------------+-------------------------+
only showing top 10 rows



## Task 3

In [42]:
# using regular expression, extract the pickup hour
taxiDF = taxiDF.withColumn('pickup_hour', func.regexp_extract(col('pickup_datetime'), '(.)([0-9]{2}):(.)', 2))

In [43]:
taxiDF_by_hour = taxiDF.groupBy('pickup_hour')\
.agg(func.sum('surcharge').alias('total_surcharge'),\
     func.sum('trip_distance').alias('total_distance'))

In [44]:
taxiDF_by_hour.show()

+-----------+---------------+------------------+
|pickup_hour|total_surcharge|    total_distance|
+-----------+---------------+------------------+
|         07|            0.0|226322.13999999972|
|         15|            9.5| 304992.9999999997|
|         11|            1.0|244987.90000000005|
|         01|        23359.5|157744.13000000027|
|         22|        50954.5| 312854.0799999997|
|         16|        59141.0|273484.30000000016|
|         18|        92658.5| 339670.9699999995|
|         00|        32440.5|         216450.19|
|         17|        74738.5|309579.85000000027|
|         09|            0.0| 242875.2900000002|
|         05|         9521.5|103116.99999999997|
|         19|        93973.5|335863.16000000003|
|         23|        39627.5|         257190.42|
|         08|            0.0|250561.12999999986|
|         03|        14707.5|104491.94999999997|
|         02|        18976.5| 124523.3799999999|
|         06|            1.5|         155507.33|
|         20|       

In [45]:
taxiDF_by_hour = taxiDF_by_hour.withColumn("Profit_Ratio", func.expr("total_surcharge/total_distance"))

In [46]:
taxiDF_by_hour.select("pickup_hour", "Profit_Ratio").sort("Profit_Ratio", ascending = False).show()

+-----------+--------------------+
|pickup_hour|        Profit_Ratio|
+-----------+--------------------+
|         19|  0.2797969863679005|
|         18| 0.27278898753108083|
|         17| 0.24141913629068537|
|         16| 0.21625007358740508|
|         20|  0.1736676383939503|
|         21|  0.1661948774517453|
|         22| 0.16286985932866863|
|         23| 0.15407844506805501|
|         02| 0.15239306867513566|
|         00| 0.14987512831474067|
|         01|  0.1480847496512229|
|         03|  0.1407524694486035|
|         04| 0.11624931403920369|
|         05| 0.09233686007156922|
|         15|3.114825586160997E-5|
|         06|9.645847562298189E-6|
|         12|5.773760950899935E-6|
|         11|4.081834245691317E-6|
|         09|                 0.0|
|         10|                 0.0|
+-----------+--------------------+
only showing top 20 rows

